NN-descent 의 Python 구현체는 pynndescent 가 있습니다 (https://github.com/lmcinnes/pynndescent/blob/master/pynndescent/pynndescent_.py)

```
pip install pynndescent
```

In [1]:
import config
import sklearn
from navernews_10days import get_bow, get_news_paths
from soynlp.utils import DoublespaceLineCorpus

x, idx_to_vocab, vocab_to_idx = get_bow(date='2016-10-20', tokenize='noun')
path = get_news_paths(date='2016-10-20')
docs = [doc for doc in DoublespaceLineCorpus(path, iter_sent=False)]

print(x.shape)

soynlp=0.0.492
added lovit_textmining_dataset
(30091, 9774)


버그가 존재합니다. Zero vector 가 포함되어 있거나, sparse matrix 이거나, 입력데이터의 차원이 크면 indexing 이 실패할 경우들이 있습니다. 일단 Random Projection 기반 인덱서에 버그가 있는 것 까지는 확인했습니다만, 이 패키지를 뜯어보지 않아서 이유는 잘 모르겠습니다.

nonzero dense matrix 로 변환하여 학습을 진행하였습니다.

In [2]:
import numpy as np

doc_length = np.asarray(x.sum(axis=1)).reshape(-1)
nonzero_doc_idxs = np.where(doc_length > 0)[0]
x_ = x[nonzero_doc_idxs].todense()

Random Projection 을 이용하여 nearest neighbor graph 를 생성합니다.

In [3]:
%%time

from pynndescent import NNDescent

index = NNDescent(x_, verbose=True, metric='cosine')

Sat Jun 22 08:20:03 2019 Building RP forest with 8 trees
Sat Jun 22 08:20:15 2019 NN descent for 10 iterations
	 0  /  10
	 1  /  10
	 2  /  10
	 3  /  10
	 4  /  10
	 5  /  10
	 6  /  10
	 7  /  10
	 8  /  10
	 9  /  10
CPU times: user 40.6 s, sys: 4.92 s, total: 45.5 s
Wall time: 45.4 s


앞선 튜토리얼들과 같은 미국 대선 문서에 대하여 similarity search 를 수행합니다.

In [4]:
query = 15
query_vec = x[query].reshape(1,-1).todense()

In [5]:
%%time

idxs, dist = index.query(query_vec, k=10)

CPU times: user 6.87 s, sys: 24 ms, total: 6.9 s
Wall time: 3.18 s


nonzero documents 만 학습에 이용하였으니 x_ 의 row index 를 원래의 row index 로 변환합니다.

In [6]:
for d, idx in zip(dist[0], idxs[0]):
    idx = nonzero_doc_idxs[idx]
    print('[idx = {}, cos = {:.3}] {} ...\n'.format(idx, 1 - d, docs[idx][:200]))

[idx = 15, cos = 1.0] 클린턴 득표율 50 목표 가능성 아주 크다  워싱턴 연합뉴스 강영두 특파원 미국 민주당 대선후보 힐러리 클린턴이 다음달 8일 대선에서 400명 이상의 선거인단을 확보하면서 대승을 거둘 수 있다는 분석이 나왔다  미국 민주당 대선후보 힐러리 클린턴이 지난 10일 현지시간 오하이오 주 콜럼버스에서 유세하는 모습 연합뉴스  미 공화당 전략가인 스티브 슈미트는 1 ...

[idx = 15006, cos = 0.846] 미국 민주당 대선후보 힐러리 클린턴이 다음달 8일 대선에서 400명 이상의 선거인단을 확보하면서 대승을 거둘 수 있다는 분석이 나왔다  미 공화당 전략가인 스티브 슈미트는 현지시간 19일 방송의 모닝조 프로그램 인터뷰에서 이 같이 전망했다 슈미트는 2008년 존 매케인의 대선캠프에 관여했다 간접선거인 미 대선에서 승리에 필요한 선거인단 매직 넘버 는 전체  ...

[idx = 7421, cos = 0.74] 여론조사서는 304명 확보 예상  힐러리 클린턴 미국 민주당 대선후보가 다음달 8일 대선에서 과반은 물론 선거인단의 4분의 3 이상 확보할 것이라는 분석이 공화당에서 나왔다  미 공화당 전략가인 스티브 슈미트는 19일 현지시간 방송과의 인터뷰에서 이같이 전망했다 그는 지난 2008년 대선 당시 공화당 후보였던 존 메케인 캠프에 관여한 인물이다  그는 현 상 ...

[idx = 11309, cos = 0.698] 공화 상 하원선거도 고전할 듯 2016년 미국 대선이 19일 앞으로 다가온 가운데 민주당 후보 힐러리 클린턴 전 국무장관이 400명 이상의 선거인단을 확보해 압승을 거둘 것이라는 전망이 제기됐다 미 공화당 전략가인 스티브 슈밋은 19일 방송의 모닝조 프로그램에 출연 클린턴이 당선 매직넘버인 선거인단 270명을 훌쩍 넘어선 400명을 확보해 공화당 도널드 트 ...

[idx = 13752, cos = 0.632] 미 대선 3주 앞 판세 분석  미국 대선을 3주일 앞두고 치러진 19일 현지시간 의 마